In [3]:
import pyspark

sc = pyspark.SparkContext('local[*]')

fruit_list = [
("apple",4),
("cherry",1),
("apple",1),
("apple",1),
("cherry",4),
("apple",1),
("cherry",3),
("apple",4),
("cherry",4),
("cherry",2),
("apple",5),
("apple",5),
("cherry",4),
("banana",2),
("apple",1),
("cherry",3),
("cherry",5),
("banana",2),
("cherry",1),
("cherry",2),
("apple",4),
("cherry",2),
("cherry",2),
("cherry",1),
("cherry",4)]

22/06/21 14:22:34 WARN Utils: Your hostname, DESKTOP-8ASBMTF resolves to a loopback address: 127.0.1.1; using 172.20.184.225 instead (on interface eth0)
22/06/21 14:22:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/21 14:22:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [23]:
fruit_rdd = sc.parallelize(fruit_list)
print("Fruit RDD: ", fruit_rdd.collect())
# print(f"It has {fruit_rdd.count()} elements")


Fruit RDD:  [('apple', 4), ('cherry', 1), ('apple', 1), ('apple', 1), ('cherry', 4), ('apple', 1), ('cherry', 3), ('apple', 4), ('cherry', 4), ('cherry', 2), ('apple', 5), ('apple', 5), ('cherry', 4), ('banana', 2), ('apple', 1), ('cherry', 3), ('cherry', 5), ('banana', 2), ('cherry', 1), ('cherry', 2), ('apple', 4), ('cherry', 2), ('cherry', 2), ('cherry', 1), ('cherry', 4)]


In [27]:
#Find the total number of rows
num_rows = fruit_rdd.count()
print("There are", num_rows,"total rows")

There are 25 total rows


In [43]:
# Find the sum of the numbers
nums = fruit_rdd.values()
sum_nums = nums.reduce(lambda x,y: x+y)
print("Sum of values:", sum_nums, "fruits")

Sum of values: 68 fruits


In [44]:
# filter to only the apple rows
apples_only = fruit_rdd.filter(lambda x: x[0] == "apple")
print(apples_only.collect())

[('apple', 4), ('apple', 1), ('apple', 1), ('apple', 1), ('apple', 4), ('apple', 5), ('apple', 5), ('apple', 1), ('apple', 4)]


In [30]:
#Group by fruit key, sum numbers:
print(fruit_rdd.reduceByKey(lambda a, b: a+b).collect())

group_fruit = fruit_rdd.groupByKey()
sum_num = group_fruit.mapValues(sum)
sum_num.collect()

[('apple', 26), ('cherry', 38), ('banana', 4)]


In [45]:
# Find the average of the second values.
avg_of_values = nums.mean()
print(avg_of_values)

2.72


In [60]:
# Return the average, sorted by fruit name.
fruit_list_sum = fruit_rdd.groupByKey().mapValues(lambda x: sum(x)/len(x)).collect()
print(fruit_list_sum)

[('apple', 2.888888888888889), ('cherry', 2.7142857142857144), ('banana', 2.0)]


1. Read the CSV into a RDD.
2. Split and map that RDD into a Map

In [62]:
import csv
from io import StringIO

color_file = "./data/colors.csv"
txt = sc.textFile(color_file)

def row_split(text_row):
    reader = csv.reader(StringIO(text_row), delimiter=',')
    for row_list in reader:
        return row_list
      
def dict_map(color_list):
    return (color_list[0],
    {'name',color_list[1],
    'rgb', color_list[2],
    'is_trans', color_list[3]
    })

colors_dict = txt.map(row_split).map(dict_map)
colors_dict.take(10)

[('id', {'is_trans', 'name', 'rgb'}),
 ('-1', {'0033B2', 'Unknown', 'f', 'is_trans', 'name', 'rgb'}),
 ('0', {'05131D', 'Black', 'f', 'is_trans', 'name', 'rgb'}),
 ('1', {'0055BF', 'Blue', 'f', 'is_trans', 'name', 'rgb'}),
 ('2', {'237841', 'Green', 'f', 'is_trans', 'name', 'rgb'}),
 ('3', {'008F9B', 'Dark Turquoise', 'f', 'is_trans', 'name', 'rgb'}),
 ('4', {'C91A09', 'Red', 'f', 'is_trans', 'name', 'rgb'}),
 ('5', {'C870A0', 'Dark Pink', 'f', 'is_trans', 'name', 'rgb'}),
 ('6', {'583927', 'Brown', 'f', 'is_trans', 'name', 'rgb'}),
 ('7', {'9BA19D', 'Light Gray', 'f', 'is_trans', 'name', 'rgb'})]